In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt

In [ ]:
# === 1. Configuration ===
IMG_SIZE = 224
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS_HEAD = 5
EPOCHS_FINE = 20
LEARNING_RATE_HEAD = 1e-3
LEARNING_RATE_FINE = 1e-4

In [ ]:
# Directories
train_dir = '/content/drive/MyDrive/Colab Notebooks/final project stanford car fine tuning/car_data/train'
test_dir = '/content/drive/MyDrive/Colab Notebooks/final project stanford car fine tuning/car_data/test'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === 2. Load datasets ===
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    label_mode='categorical',
    shuffle=True
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    label_mode='categorical',
    shuffle=False
)

num_classes = len(train_ds.class_names)
print(f"Detected {num_classes} classes.")

Found 12064 files belonging to 196 classes.
Found 4121 files belonging to 196 classes.
Detected 196 classes.


In [ ]:
# Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# === 3. Preprocessing + Augmentation ===
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def process(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [ ]:
# === 4. Build model ===
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze the convolutional base

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_HEAD),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 196)            │       251,076 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,509,060 (9.57 MB)

 Trainable params: 251,076 (980.77 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# === 5. Train classifier head ===
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD
)

Epoch 1/5
754/754 ━━━━━━━━━━━━━━━━━━━━ 1908s 3s/step - accuracy: 0.0628 - loss: 4.9367 - val_accuracy: 0.2041 - val_loss: 3.4656
Epoch 2/5
754/754 ━━━━━━━━━━━━━━━━━━━━ 1025s 1s/step - accuracy: 0.2689 - loss: 3.1768 - val_accuracy: 0.2584 - val_loss: 3.1240
Epoch 3/5
754/754 ━━━━━━━━━━━━━━━━━━━━ 1005s 1s/step - accuracy: 0.3595 - loss: 2.7105 - val_accuracy: 0.3021 - val_loss: 2.9167
Epoch 4/5
754/754 ━━━━━━━━━━━━━━━━━━━━ 1039s 1s/step - accuracy: 0.4071 - loss: 2.4565 - val_accuracy: 0.3215 - val_loss: 2.8056
Epoch 5/5
754/754 ━━━━━━━━━━━━━━━━━━━━ 1013s 1s/step - accuracy: 0.4439 - loss: 2.2687 - val_accuracy: 0.3281 - val_loss: 2.7754


In [ ]:
# === 6. Fine-tune top layers ===
base_model.trainable = True

# Freeze all but last N layers, or keep batch-norm layers frozen
fine_tune_at = len(base_model.layers) - 50
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD + EPOCHS_FINE,
    initial_epoch=history1.epoch[-1]
)

Epoch 5/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1450s 2s/step - accuracy: 0.3945 - loss: 2.5561 - val_accuracy: 0.4700 - val_loss: 2.1227
Epoch 6/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1454s 2s/step - accuracy: 0.6589 - loss: 1.2550 - val_accuracy: 0.5637 - val_loss: 1.6862
Epoch 7/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1413s 2s/step - accuracy: 0.7414 - loss: 0.9159 - val_accuracy: 0.6013 - val_loss: 1.5640
Epoch 8/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1462s 2s/step - accuracy: 0.7982 - loss: 0.7025 - val_accuracy: 0.5977 - val_loss: 1.5789
Epoch 9/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1416s 2s/step - accuracy: 0.8289 - loss: 0.5720 - val_accuracy: 0.6423 - val_loss: 1.3951
Epoch 10/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1366s 2s/step - accuracy: 0.8710 - loss: 0.4346 - val_accuracy: 0.6392 - val_loss: 1.4317
Epoch 11/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1415s 2s/step - accuracy: 0.8935 - loss: 0.3663 - val_accuracy: 0.6841 - val_loss: 1.1944
Epoch 12/25
754/754 ━━━━━━━━━━━━━━━━━━━━ 1361s 2s/step - accuracy: 0.9086 - loss: 0.3216

In [ ]:
# === 7. Save model ===
model.save('mobilenetv2_stanford_cars.h5')
print("Model trained and saved.")



In [ ]:
# === 8. Evaluate ===
loss, acc = model.evaluate(val_ds)
print(f"Final accuracy: {acc:.4f}")

In [ ]:
# === 9. Save model ===
model.save('mobilenetv2_stanford_cars.keras')
print("Model trained and saved.")

In [ ]:
import os
from tensorflow.keras.models import load_model
from google.colab import files

save_dir = "/content/drive/MyDrive/Colab Notebooks/final project stanford car fine tuning"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, "model.keras")
model.save(model_path)

In [ ]:


# === 9. Plot training curves ===
def plot_training_history(history_list):
    """
    Plots training & validation accuracy/loss from a list of Keras History objects.
    Useful if training was done in stages.
    """
    acc = []
    val_acc = []
    loss = []
    val_loss = []

    for hist in history_list:
        acc += hist.history['accuracy']
        val_acc += hist.history['val_accuracy']
        loss += hist.history['loss']
        val_loss += hist.history['val_loss']

    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()


In [ ]:
plot_training_history([history1, history2])
